# Réseaux neuronaux

<img src="images/mnist-examples.png"/>

Quelques guides:
- keras https://keras.io/getting_started/intro_to_keras_for_engineers/
- keras https://www.tensorflow.org/tutorials/quickstart/beginner
- (tf2) https://becominghuman.ai/image-classification-with-tensorflow-2-0-without-keras-e6534adddab2
- (tf1) https://github.com/neodelphis/tensorflow-without-a-phd-french/blob/master/mnist_03_softmax.ipynb

## Softmax

Créer et mesurer les résultats d'un réseau neuronal simple couche complètement connecté avec softmax comme fonction d'activation

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

Récupération des données sous forme de tableaux numpy

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
# Dimension de l'ensemble d'entraînement des images:
x_train.shape

(60000, 28, 28)

In [4]:
# Dimension de l'ensemble d'entraînement des étiquettes:
y_test.shape

(10000,)

Travail préparatoire sur les données d'entrée

In [5]:
# normalisation
x_train, x_test = x_train/255., x_test/255.
# 1-hot encoding
y_train, y_test = to_categorical(y_train, dtype='int8'), to_categorical(y_test, dtype='int8')

Définition de l'architecture du réseau de neurones : modèle simple couche complètement connecté avec softmax comme fonction d'activation

In [6]:
inputs = tf.keras.Input(shape=(28, 28))
x = tf.keras.layers.Flatten()(inputs)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="mono_couche_softmax")
model.summary()

Model: "mono_couche_softmax"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


Choix de la fonction de coût, de la stratégie d'optimisation et des mesures

In [7]:
# Compile the model
model.compile(
    optimizer="sgd",
    loss="categorical_crossentropy",
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="acc")],
)

Entraînement du modèle

In [10]:
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=5)

Fit on NumPy data
Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 1s 24us/sample - loss: 0.4370 - acc: 0.8841
Epoch 2/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.4125 - acc: 0.8888
Epoch 3/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3954 - acc: 0.8922
Epoch 4/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3825 - acc: 0.8950
Epoch 5/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3723 - acc: 0.8976


Evaluation sur l'ensemble de test

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 0s - loss: 0.2819 - acc: 0.9056


[0.3499122075915337, 0.9056]

## Sigmoïde

Créer et mesurer les résultats d'un réseau neuronal complètement connecté:
- première couche 100 neurones avec sigmoïde comme fonction d'activation
- deuxième couche de 10 neurones avec softmax comme fonction d'activation